<a href="https://colab.research.google.com/github/Rohan2390/AVENDI_PYTORCH/blob/main/neural_nets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#imports
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F # relu,tanh,etc
from torch.utils.data import DataLoader # easier dataset managememt (minibatches)
import torchvision.datasets as Datasets
import torchvision.transforms as transforms #used for transform ationms to perform on datsets

In [6]:
#create a fully connected network
class NN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(NN, self).__init__()
        # Our first linear layer take input_size, in this case 784 nodes to 50
        # and our second linear layer takes 50 to the num_classes we have, in
        # this case 10.
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, num_classes)

    def forward(self, x):
        """
        x here is the mnist images and we run it through fc1, fc2 that we created above.
        we also add a ReLU activation function in between and for that (since it has no parameters)
        I recommend using nn.functional (F)
        """

        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [9]:
#check whether it is working or not
model = NN(784,10)
x = torch.randn(64,784)
model(x).shape

torch.Size([64, 10])

In [10]:
#set device
device = torch.device('cuda' if torch.cuda.is_available()else 'cpu')

In [11]:
#hyperparameters
input_size = 784
num_classes = 10
batch_size = 64
lr = 0.001
num_epochs = 100

In [18]:
#load data
train_dataset = Datasets.MNIST(root = '/content/drive/MyDrive/PYTORCH_AVENDI/MNIST' , train = True,transform = transforms.ToTensor(),download = True)
test_dataset = Datasets.MNIST(root = '/content/drive/MyDrive/PYTORCH_AVENDI/MNIST' , train = False,transform = transforms.ToTensor(),download = True) 
train_loader = DataLoader(dataset = train_dataset , batch_size = batch_size , shuffle = True)
test_loader = DataLoader(dataset = test_dataset , batch_size = batch_size , shuffle = False)

In [19]:
#initialize network
model = NN(input_size = input_size , num_classes = num_classes)

In [21]:
#loss and optimizers
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters() , lr = lr)

In [24]:
#train the network
for epoch in range(num_epochs):
    for batch_idx, (data , targets) in enumerate(train_loader):

        data = data.to(device = device)
        targets = targets.to(device = device)


        data = data.reshape(data.shape[0], -1) #to flatten


        #forward prop
        forward_prop = model(data)
        loss = criterion(forward_prop , targets)

        #backward prop
        optimizer.zero_grad()
        loss.backward()

        #update by adam
        optimizer.step()



In [25]:
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            x = x.reshape(x.shape[0], -1)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

    model.train()
    return num_correct/num_samples


print(f"Accuracy on training set: {check_accuracy(train_loader, model)*100:.2f}")
print(f"Accuracy on test set: {check_accuracy(test_loader, model)*100:.2f}")

Accuracy on training set: 95.09
Accuracy on test set: 94.98
